# KNN (K-Nearest Neighbors) 算法实现

本笔记本包含 KNN 算法的完整实现和实验。

## 目录
1. [从零实现 KNN](#从零实现-knn)
2. [使用 sklearn](#使用-sklearn)
3. [交叉验证选择 K 值](#交叉验证选择-k-值)
4. [不同距离度量对比](#不同距离度量对比)

## 从零实现 KNN

首先，我们从零开始实现一个简单的 KNN 分类器。

In [ ]:
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class SimpleKNN:
    """简单的 KNN 分类器实现"""
    
    def __init__(self, k=3, distance='euclidean'):
        self.k = k
        self.distance = distance
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """KNN 不需要训练，只需存储数据"""
        self.X_train = X
        self.y_train = y
        return self
    
    def _distance(self, a, b):
        if self.distance == 'euclidean':
            return np.sqrt(np.sum((a - b) ** 2))
        elif self.distance == 'manhattan':
            return np.sum(np.abs(a - b))
        else:
            raise ValueError(f"Unknown distance metric: {self.distance}")
    
    def predict(self, X):
        predictions = []
        for x in X:
            # 计算到所有训练点的距离
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            # 找到 k 个最近邻
            k_indices = np.argsort(distances)[:self.k]
            k_labels = self.y_train[k_indices]
            # 投票
            most_common = Counter(k_labels).most_common(1)[0][0]
            predictions.append(most_common)
        return np.array(predictions)
    
    def predict_proba(self, X):
        """返回预测概率"""
        probas = []
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            k_indices = np.argsort(distances)[:self.k]
            k_labels = self.y_train[k_indices]
            # 计算每个类别的比例
            counts = Counter(k_labels)
            total = sum(counts.values())
            proba = {label: count/total for label, count in counts.items()}
            probas.append(proba)
        return probas

In [ ]:
# 测试我们的实现
# 创建数据集
X, y = make_classification(
    n_samples=200, n_features=2, n_redundant=0,
    n_informative=2, random_state=42, n_clusters_per_class=1
)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 训练和预测
knn = SimpleKNN(k=3)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

# 计算准确率
accuracy = np.mean(predictions == y_test)
print(f"准确率: {accuracy:.2%}")

## 使用 sklearn

生产环境中，我们使用 sklearn 的 KNN 实现。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 创建 sklearn KNN
sknn = KNeighborsClassifier(n_neighbors=3)
sknn.fit(X_train, y_train)

# 预测
y_pred = sknn.predict(X_test)

# 评估
print("分类报告:")
print(classification_report(y_test, y_pred))

print("\n混淆矩阵:")
print(confusion_matrix(y_test, y_pred))

## 交叉验证选择 K 值

K 值是 KNN 最重要的超参数，我们需要通过交叉验证来选择最佳值。

In [ ]:
from sklearn.model_selection import cross_val_score

# 测试不同的 K 值
k_values = range(1, 31, 2)
cv_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

# 找到最佳 K
best_k = k_values[np.argmax(cv_scores)]
best_score = max(cv_scores)

print(f"最佳 K 值: {best_k}")
print(f"最佳交叉验证得分: {best_score:.2%}")

# 绘制 K 值 vs 准确率
plt.figure(figsize=(10, 6))
plt.plot(k_values, cv_scores, marker='o', linestyle='-')
plt.axvline(x=best_k, color='r', linestyle='--', label=f'最佳 K={best_k}')
plt.xlabel('K 值')
plt.ylabel('交叉验证准确率')
plt.title('K 值选择')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 不同距离度量对比

KNN 支持多种距离度量，我们来对比它们的效果。

In [ ]:
# 对比不同距离度量
metrics = ['euclidean', 'manhattan', 'minkowski']
results = {}

for metric in metrics:
    knn = KNeighborsClassifier(n_neighbors=best_k, metric=metric)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    results[metric] = scores.mean()
    print(f"{metric}: {scores.mean():.2%} (+/- {scores.std():.2%})")

## 可视化决策边界

让我们可视化 KNN 的决策边界。

In [ ]:
def plot_decision_boundary(X, y, model, h=0.02):
    """绘制决策边界"""
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(f'KNN 决策边界 (K={model.n_neighbors})')
    plt.show()

# 绘制决策边界
plot_decision_boundary(X_train, y_train, sknn)

## 总结

**KNN 的优缺点：**

### 优点
1. 简单易理解
2. 无需训练过程
3. 对异常值相对鲁棒（当 K > 1）

### 缺点
1. 预测需要遍历所有数据，速度慢
2. 对高维数据效果差（维度诅咒）
3. 需要选择合适的 K 值和距离度量

**优化技巧：**
- 使用 KD-Tree 或 Ball-Tree 加速近邻搜索
- 对特征进行标准化
- 使用交叉验证选择最佳 K 值
- 对于高维数据，考虑降维（如 PCA）